# Taking Interleaved Thinking to the Next Level: Travel Planning

Now that we've explored the fundamentals of interleaved thinking, let's dive into a more complex, real-world application that really showcases the power of this capability. When we first implemented interleaved thinking, travel planning immediately jumped out as one of those perfect use cases - a complex, multi-step process with interdependent decisions that requires integrating diverse information.

In this advanced example, we'll use Claude's interleaved thinking to plan a family trip to Japan. This is a much more intricate reasoning chain than our revenue calculation, involving flight searches, weather forecasts, accommodation recommendations, attraction suggestions, and ultimately creating a comprehensive itinerary.

## Importing from utils.py and Setting Up

First, let's import our tools from utils.py and configure our model:

In [ ]:
!pip install boto3 botocore awscli --upgrade --quiet

In [ ]:
# Import necessary libraries
import boto3
import json
import time
import copy
import uuid
from botocore.config import Config
from datetime import datetime, timedelta

REGION_NAME="us-west-2"

# Configure Bedrock client with appropriate region
config = Config(
    region_name=REGION_NAME,  # Change to your preferred region
    retries={
        'max_attempts': 15,
        'mode': 'adaptive'
    }
)

# Initialize Bedrock Runtime client
bedrock_rt = boto3.client('bedrock-runtime', config=config)

print("Bedrock client initialized successfully!")

In [ ]:
# Import tool execution functions from utils.py
from utils import (
    execute_tool, 
    search_flights,
    search_accommodations,
    get_weather_forecast,
    search_attractions,
    create_itinerary
)

print("Tools imported successfully!")

In [ ]:
# Select the appropriate Claude 4 model
# MODEL_ID = "us.anthropic.claude-opus-4-20250514-v1:0"  # Larger, more capable model
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"  # Balanced model for most use cases

print("Utility functions for tools imported successfully!")
print(f"Using model: {MODEL_ID}")

## The Travel Tools in Our Toolkit

For this travel planning example, we'll need a more extensive set of tools than our earlier revenue calculation example. These tools simulate what you might find in a real travel planning application:

In [ ]:
# Configure tools for the model to use
tools = [
    {
        "toolSpec": {
            "name": "search_flights",
            "description": "Search for flights between destinations",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Origin airport or city code"},
                        "destination": {"type": "string", "description": "Destination airport or city code"},
                        "date": {"type": "string", "description": "Travel date in YYYY-MM-DD format"}
                    },
                    "required": ["origin", "destination", "date"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "search_accommodations",
            "description": "Search for accommodations in a location",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City or region name"},
                        "check_in": {"type": "string", "description": "Check-in date in YYYY-MM-DD format"},
                        "check_out": {"type": "string", "description": "Check-out date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "check_in", "check_out"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "get_weather_forecast",
            "description": "Get weather forecast for a location and date",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City or region name"},
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format"}
                    },
                    "required": ["location", "date"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "search_attractions",
            "description": "Search for attractions in a location",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City or region name"},
                        "category": {"type": "string", "description": "Category of attractions (Cultural, Family, Shopping, Nature, Food)"}
                    },
                    "required": ["location"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "create_itinerary",
            "description": "Create a travel itinerary",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "days": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "date": {"type": "string"},
                                    "location": {"type": "string"},
                                    "activities": {"type": "array", "items": {"type": "object"}}
                                }
                            }
                        }
                    },
                    "required": ["days"]
                }
            }
        }
    }
]

print("Travel planning tools defined!")

## The Power of Reflection - Our Advanced Interleaved Thinking Function

Now, let's implement our advanced interleaved thinking function that demonstrates a sophisticated travel planning process. This function really showcases what interleaved thinking is capable of when pushed to its limits!

In [ ]:
def demonstrate_interleaved_thinking(prompt):
    """
    Demonstrate Claude's interleaved thinking capabilities with advanced travel planning,
    showing each step of the process in real-time as it happens.
    """
    #--------------------------------------------------
    # SECTION 1: Setup and initialization
    #--------------------------------------------------
    # Print header
    print("\n" + "=" * 100)
    print("🧠 INTERLEAVED THINKING DEMONSTRATION: TRIP PLANNING WITH CLAUDE 🧠".center(100))
    print("=" * 100 + "\n")
    print("This demonstration shows Claude's interleaved thinking capabilities in real-time.")
    print("You'll see Claude's reasoning process, tool calls, and how it builds a travel plan step by step.\n")
    
    # Initialize conversation
    conversation = []

    # Enhance prompt to encourage thoughtful planning with reflection
    enhanced_prompt = prompt + """

As you help plan this trip, please:
1. Think carefully about what information you need at each stage of the planning process
2. Feel free to gather related information at once if appropriate (like weather for multiple cities)
3. After receiving information, reflect on what you've learned and explain how it affects your planning
4. Consider how each piece of information influences your recommendations for the trip
"""
    # Add the initial user message
    user_message = {
        "role": "user", 
        "content": [{"text": enhanced_prompt}]
    }
    conversation.append(user_message)
    
    # Print the initial prompt
    print("📝 INITIAL PLANNING REQUEST:")
    print("-" * 80)
    print(prompt)
    print("-" * 80 + "\n")
    
    # Configure tools
    tool_config = {"tools": tools}
    
    # Process continues until Claude stops requesting tools
    planning_complete = False
    iteration = 1
    
    #--------------------------------------------------
    # SECTION 2: Main interleaved thinking loop
    #--------------------------------------------------
    while not planning_complete:
        print(f"\n📊 PLANNING ITERATION #{iteration}")
        print("=" * 80 + "\n")
        
        # Call Claude with both extended thinking and interleaved thinking
        print("Calling Claude API with interleaved thinking enabled...\n")
        
        try:
            response = bedrock_rt.converse(
                modelId=MODEL_ID,
                messages=conversation,
                toolConfig=tool_config,
                additionalModelRequestFields={
                    "anthropic_beta": ["token-efficient-tools-2025-02-19",
                                       "interleaved-thinking-2025-05-14"],
                    "max_tokens": 20000,
                    "reasoning_config": {
                        "type": "enabled", 
                        "budget_tokens": 16000
                    }
                }
            )
            
            #--------------------------------------------------
            # SECTION 3: Processing Claude's response
            #--------------------------------------------------
            # Process the assistant's response
            if "output" in response and "message" in response["output"]:
                assistant_message = response["output"]["message"]
                
                # Add to the conversation history
                conversation.append(assistant_message)
                
                # Extract and display reasoning content
                has_reasoning = False
                has_tool_calls = False
                tool_calls = []
                assistant_text = ""
                
                # Process each content item
                for content_item in assistant_message.get("content", []):
                    # Handle reasoning (extended thinking)
                    if "reasoningContent" in content_item:
                        has_reasoning = True
                        reasoning_item = content_item["reasoningContent"].get("reasoningText", {})
                        if "text" in reasoning_item:
                            reasoning_text = reasoning_item["text"]
                            
                            # Display the reasoning
                            print("🧠 CLAUDE'S EXTENDED THINKING:")
                            print("~" * 80)
                            print(reasoning_text)
                            print("~" * 80)
                            print("\nℹ️ Above, you can see Claude's internal reasoning process as it analyzes")
                            print("   the planning problem and decides what information it needs.\n")
                    
                    # Handle regular text content
                    elif "text" in content_item:
                        assistant_text += content_item["text"]
                    
                    # Handle tool use
                    elif "toolUse" in content_item:
                        has_tool_calls = True
                        tool_use = content_item["toolUse"]
                        tool_name = tool_use.get("name", "unknown-tool")
                        tool_input = tool_use.get("input", {})
                        tool_use_id = tool_use.get("toolUseId")
                        
                        tool_calls.append({
                            "name": tool_name,
                            "input": tool_input,
                            "id": tool_use_id
                        })
                
                # Display Claude's response text
                if assistant_text:
                    print("🔍 CLAUDE'S PLANNING PROCESS:")
                    print("-" * 80)
                    print(assistant_text)
                    print("-" * 80)
                    print("\nℹ️ This shows Claude's visible thinking as it works through the planning process.\n")
                
                #--------------------------------------------------
                # SECTION 4: Loop control, tool execution and reflection
                #--------------------------------------------------
                # If no tool calls, planning is complete
                if not has_tool_calls:
                    print("\n✅ PLANNING COMPLETE!")
                    print("Claude has finished the planning process and provided a complete itinerary.")
                    planning_complete = True
                    continue
                
                # Process and display tool calls
                print(f"🛠️ CLAUDE REQUESTED {len(tool_calls)} TOOL CALL(S):\n")
                
                # Process each tool call
                all_tool_results = []
                
                for tool_call in tool_calls:
                    tool_name = tool_call["name"]
                    tool_input = tool_call["input"]
                    tool_use_id = tool_call["id"]
                    
                    # Display the tool call
                    print(f"TOOL CALL: {tool_name}")
                    print(f"INPUT: {json.dumps(tool_input, indent=2)}")
                    print("-" * 80)
                    
                    try:
                        # Execute the tool
                        tool_result = execute_tool(tool_name, tool_input)
                        tool_result_json = json.dumps(tool_result, indent=2)
                        
                        # Display the result
                        print(f"RESULT:")
                        print(tool_result_json)
                        print("-" * 80)
                        
                        # Add to our collection of results for the API
                        all_tool_results.append({
                            "toolUseId": tool_use_id,
                            "content": [{"text": tool_result_json}],
                            "status": "success"
                        })
                        
                    except Exception as e:
                        print(f"ERROR: {str(e)}")
                        print("-" * 80)
                        
                        # Add error result
                        all_tool_results.append({
                            "toolUseId": tool_use_id,
                            "content": [{"text": f"Error: {str(e)}"}],
                            "status": "error"
                        })
                
                # If tool calls were made, add their results to the conversation
                if all_tool_results:
                    print("\nℹ️ Next, Claude will analyze these tool results and decide what to do next.\n")
                    print("   This demonstrates interleaved thinking - reasoning between tool calls.\n")
                    
                    # Create a single message with all tool results
                    tool_results_message = {
                        "role": "user",
                        "content": [
                            {"toolResult": result} for result in all_tool_results
                        ]
                    }
                    
                    # Add to conversation
                    conversation.append(tool_results_message)
                    
                    # THE KEY INNOVATION: Encouraging reflection on the tool results
                    reflection_message = {
                        "role": "user",
                        "content": [{
                            "text": f"""
Now that you have this information, please:
1. Analyze what you've learned and how it affects your planning
2. Explain your reasoning process and what you'll focus on next
3. Continue with the planning process, gathering any additional information you need
"""
                        }]
                    }
                    
                    # Add reflection prompt to conversation
                    conversation.append(reflection_message)
            
            # Increment iteration counter
            iteration += 1
            
        except Exception as e:
            print(f"ERROR: {str(e)}")
            import traceback
            traceback.print_exc()
            break
    
    # Print final summary
    print("\n" + "=" * 100)
    print("🏁 INTERLEAVED THINKING DEMONSTRATION COMPLETE 🏁".center(100))
    print("=" * 100 + "\n")
    print("You've just witnessed Claude's interleaved thinking capabilities in action.")
    print("Key aspects demonstrated include:")
    print("- Extended thinking to analyze the planning problem")
    print("- Strategic tool selection to gather necessary information")
    print("- Reflective analysis between tool calls")
    print("- Iterative planning that builds toward a complete solution")
    print("\nThis showcases how interleaved thinking enables more sophisticated")
    print("reasoning and planning than would be possible with simple tool calls.")

## Define Our Trip Planning Prompt

Now let's create our travel planning prompt that will kick off the interleaved thinking process:

In [ ]:
# Define trip planning prompt
trip_planning_prompt = """
I need your help planning a 5-day trip to Japan for a family of four (parents, 8-year-old, and 14-year-old).
We'll be traveling from Los Angeles (LAX) to Japan in June 2025, starting on June 15th.
Our budget is $6,000 for accommodations and activities (excluding flights).

Please help by:
1. Finding flight options from LAX to Tokyo for June 15th, 2025
2. Checking weather forecasts for Tokyo, Kyoto, and Osaka during our visit (June 15-19, 2025)
3. Suggesting family-friendly accommodations in these cities
4. Recommending attractions suitable for our family's age range
5. Creating a practical itinerary considering jetlag and possible rain

Use parallel tool calls to be efficient providing your response
"""

print("Trip planning prompt defined!")

## Run the Advanced Interleaved Thinking Demonstration

Let's run our travel planning demonstration to see interleaved thinking in action:

In [ ]:
# Run the demonstration
demonstrate_interleaved_thinking(trip_planning_prompt)

## The Real Magic: Adding Reflection to Interleaved Thinking

What you've just witnessed represents the true promise of interleaved thinking - not just the ability to call tools sequentially, but to create a genuinely reflective planning process that's much closer to human cognition.

### The Secret Sauce: Reflection Prompts

The most powerful innovation in this implementation is what I call the "reflection prompt." Did you spot it? After Claude receives tool results, we don't just send those results back. Instead, we add an explicit prompt:

```python
reflection_message = {
    "role": "user",
    "content": [{
        "text": f"""
Now that you have this information, please:
1. Analyze what you've learned and how it affects your planning
2. Explain your reasoning process and what you'll focus on next
3. Continue with the planning process, gathering any additional information you need
"""
    }]
}
```

This seemingly simple addition transforms the entire planning process. It encourages Claude to:

1. **Process information deeply** rather than just moving to the next action
2. **Explain its reasoning** so you can follow along with its thinking
3. **Make strategic decisions** about what information to gather next

When I first added this reflection step, I was amazed at the difference it made. Instead of a mechanical sequence of tool calls, Claude started to show genuine adaptive planning - changing course based on new information, considering trade-offs, and building a coherent plan that incorporated everything it had learned.

### Applications Beyond Travel Planning

While this example focuses on travel planning, the pattern works for virtually any complex planning or decision-making scenario:

- **Financial planning**: Gathering account information, analyzing spending patterns, projecting future needs
- **Healthcare decision support**: Researching conditions, treatment options, and outcomes
- **Project management**: Evaluating resources, timelines, and dependencies
- **Educational planning**: Assessing student needs, available resources, and curriculum options

The key is always the same: interleave tool use with explicit reflection prompts to create a richer, more adaptive reasoning process.

### Implementation Tips from the Trenches

If you're implementing this pattern yourself, here are some hard-earned tips:

1. **Design tools for composability**: Create smaller, focused tools rather than monolithic ones
2. **Craft effective reflection prompts**: Ask Claude to explain its reasoning and decision process
3. **Be generous with token budgets**: Reflective reasoning needs space to develop
4. **Log the full conversation**: The reasoning trail is invaluable for debugging and improving your system

One of my favorite debugging techniques is to review Claude's thinking across multiple iterations - you'll often see it catching its own mistakes or revising its approach based on new information, just like a human expert would.

## The Future of Interleaved Thinking

As we continue to develop this capability, I see several exciting directions:

1. **Hierarchical Planning**: Creating top-level plans that unfold into more detailed sub-plans
2. **Recursive Problem Solving**: Breaking complex problems into parts, solving each independently
3. **Extended Memory**: Enhancing the system with persistent memory across sessions
4. **Human-in-the-loop Collaboration**: Combining Claude's planning with human guidance at key decision points

The travel planning example you've just seen is just the beginning. As we incorporate these more advanced patterns, we're moving toward truly collaborative AI systems that can tackle problems with depth and sophistication previously limited to human experts.

Remember, the goal isn't to replace human thinking but to augment it - creating AI partners that can handle routine parts of complex tasks while collaborating effectively with humans on the most nuanced decisions.

## Appendix - Simplified Implementation for Your Projects

If you want to incorporate this advanced interleaved thinking with reflection into your own projects, here's a simplified version that maintains the core functionality while removing the verbose logging and display code:

In [ ]:
# Select the appropriate Claude 4 model
# MODEL_ID = "us.anthropic.claude-opus-4-20250514-v1:0"  # Larger, more capable model
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"  # Balanced model for most use cases

print(f"Using Claude 4 model: {MODEL_ID}")

In [ ]:
def simplified_travel_planner(prompt, tools):
    """
    A simplified implementation of interleaved thinking with reflection prompts
    for travel planning or other complex reasoning tasks.
    
    This version maintains the core functionality while removing the detailed
    logging and display code for easier integration into your own projects.
    """
    # Initialize conversation with enhanced prompt for reflection
    enhanced_prompt = prompt + """
    
As you work through this planning process:
1. Carefully analyze information at each step
2. Explain how new information influences your decisions
3. Consider alternatives and explain your reasoning
"""
    # Set up conversation
    conversation = [{"role": "user", "content": [{"text": enhanced_prompt}]}]
    tool_config = {"tools": tools}
    
    # Main planning loop
    planning_complete = False
    
    while not planning_complete:
        # Call Claude with interleaved thinking enabled
        response = bedrock_rt.converse(
            modelId=MODEL_ID,
            messages=conversation,
            toolConfig=tool_config,
            additionalModelRequestFields={
                "anthropic_beta": ["token-efficient-tools-2025-02-19",
                                   "interleaved-thinking-2025-05-14"],
                "max_tokens": 4000,
                "reasoning_config": {
                    "type": "enabled", 
                    "budget_tokens": 3000
                }
            }
        )
        
        # Get assistant's message and add to conversation
        assistant_message = response["output"]["message"]
        conversation.append(assistant_message)
        
        # Check for tool calls
        has_tool_calls = False
        tool_results = []
        
        # Process each content item for tool calls
        for content_item in assistant_message.get("content", []):
            if "toolUse" in content_item:
                has_tool_calls = True
                tool_use = content_item["toolUse"]
                tool_name = tool_use["name"]
                tool_input = tool_use["input"]
                
                # Print what's happening - similar to the database and calculator logs
                if tool_name == "get_weather_forecast":
                    print(f"Checking weather forecast for {tool_input.get('location')} on {tool_input.get('date')}")
                elif tool_name == "search_accommodations":
                    print(f"Searching for accommodations in {tool_input.get('location')} from {tool_input.get('check_in')} to {tool_input.get('check_out')}")
                elif tool_name == "search_attractions":
                    if "category" in tool_input and tool_input.get('category'):
                        print(f"Searching for {tool_input.get('category')} attractions in {tool_input.get('location')}")
                    else:
                        print(f"Searching for all attractions in {tool_input.get('location')}")
                elif tool_name == "search_flights":
                    print(f"Searching for flights from {tool_input.get('origin')} to {tool_input.get('destination')} on {tool_input.get('date')}")
                elif tool_name == "create_itinerary":
                    print(f"Creating itinerary with {len(tool_input.get('days', []))} days")
                else:
                    print(f"Executing {tool_name}: {json.dumps(tool_input)}")
                    
                # Execute tool
                result = execute_tool(tool_name, tool_input)
                
                # Add to results
                tool_results.append({
                    "toolUseId": tool_use["toolUseId"],
                    "content": [{"text": json.dumps(result)}],
                    "status": "success"
                })
        
        # If no tool calls, we're done
        if not has_tool_calls:
            return assistant_message
        
        # Add tool results and continue loop
        tool_results_message = {
            "role": "user",
            "content": [{"toolResult": result} for result in tool_results]
        }
        conversation.append(tool_results_message)
        
        # KEY FEATURE: Add reflection prompt
        reflection_message = {
            "role": "user",
            "content": [{
                "text": """
Now that you have this information, please:
1. Analyze what you've learned and how it affects your planning
2. Explain your reasoning process and what you'll focus on next
3. Continue with the planning process, gathering any additional information you need
"""
            }]
        }
        conversation.append(reflection_message)
    
    return "Planning process failed to complete"

def extract_plan_from_message(message):
    """Extract the final plan text from Claude's message"""
    plan_text = ""
    for content_item in message.get("content", []):
        if "text" in content_item:
            plan_text += content_item["text"]
    
    return plan_text

This simplified implementation is perfect for integrating into your own projects. I've stripped away all the detailed logging and display code while preserving the core interleaved thinking pattern with reflection prompts.
The key elements I've maintained are:

1. The enhanced prompt that encourages reflective thinking
2. The core conversation loop with interleaved thinking enabled
3. Tool call extraction and execution
4. The critical reflection prompt after tool results
5. Final plan extraction from Claude's response

This pattern can be adapted for virtually any complex reasoning task - not just travel planning. The reflection prompts are the secret ingredient that elevates this from a simple tool-calling implementation to a genuinely reflective planning process.
## Running the Simplified Travel Planner
Now, let's see how to execute our simplified travel planner with a new example query:

In [ ]:
# A new example trip planning request
weekend_trip_request = """
Please help me plan a weekend trip to San Francisco for a couple celebrating their anniversary.
We're looking for romantic dining options, some sightseeing, and possibly a special experience.
We'll be visiting August 12-14, 2025, arriving Friday evening and departing Sunday afternoon.
Our budget is moderate but we're willing to splurge on one special meal or experience.
"""

print("Planning a weekend getaway to San Francisco...")
print("-" * 80)

# Call the simplified_travel_planner function directly
print("Running simplified travel planner...\n")
result = simplified_travel_planner(weekend_trip_request, tools)

# Display the final result
print("\n" + "=" * 80)
print("WEEKEND GETAWAY PLAN:".center(80))
print("=" * 80)
# Extract and print just the text content from the response
for content_item in result.get("content", []):
    if "text" in content_item:
        print(content_item["text"])

When you run this cell, you'll see the simplified planner in action. Unlike our detailed demonstration, this version doesn't show all the intermediate steps, thinking processes, and tool calls. Instead, it works quietly behind the scenes and returns just the final plan.

This approach is perfect for production applications where you want to provide a clean, polished experience to end users without exposing all the "behind the scenes" reasoning and tool usage.
What I love about this simplified implementation is how it preserves the powerful reflection capabilities while removing all the logging and display code. It's like having a seasoned travel agent working behind the scenes - you don't see all their research and deliberation, just the final, well-crafted itinerary.

Feel free to customize this implementation for your specific use case. You might adjust the reflection prompts, modify the tool execution logic, or add additional processing of Claude's responses to extract structured data.
Happy building! 🚀